In [ ]:
import os
import sys
import os.path
from sys import platform
from pathlib import Path
import re
import sys
import time
import copy
import math
import scipy
import torch
import pickle
import random
import argparse
import subprocess
import sklearn
import numpy as np
import pandas as pd
from torch import nn
from torch.utils import data
from torch.nn.utils.weight_norm import weight_norm
from sklearn import datasets
from sklearn import preprocessing
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn import metrics
from sklearn import preprocessing
from sklearn.metrics import auc
from sklearn.metrics import r2_score
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import StandardScaler
from typing import Union, List, Tuple, Sequence, Dict, Any, Optional, Collection
import seaborn as sns
from scipy import stats
import matplotlib.pyplot as plt
from pathlib import Path
from copy import deepcopy
from ipywidgets import IntProgress
from datetime import datetime
from mpl_toolkits.axes_grid1 import make_axes_locatable
from torch.utils.data import Dataset, DataLoader
from torch.utils.tensorboard import SummaryWriter

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive/')

Mounted at /content/gdrive/


In [ ]:
import sys
sys.path.append('/content/gdrive/MyDrive/function_predictor/code/')

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
class StandardScaler:
    """A :class:`StandardScaler` normalizes the features of a dataset.

    When it is fit on a dataset, the :class:`StandardScaler` learns the mean and standard deviation across the 0th axis.
    When transforming a dataset, the :class:`StandardScaler` subtracts the means and divides by the standard deviations.
    """

    def __init__(self, means: np.ndarray = None, stds: np.ndarray = None, replace_nan_token: Any = None):
        """
        :param means: An optional 1D numpy array of precomputed means.
        :param stds: An optional 1D numpy array of precomputed standard deviations.
        :param replace_nan_token: A token to use to replace NaN entries in the features.
        """
        self.means = means
        self.stds = stds
        self.replace_nan_token = replace_nan_token

    def fit(self, X: List[List[Optional[float]]]) -> 'StandardScaler':
        """
        Learns means and standard deviations across the 0th axis of the data :code:`X`.

        :param X: A list of lists of floats (or None).
        :return: The fitted :class:`StandardScaler` (self).
        """
        X = np.array(X).astype(float)
        self.means = np.nanmean(X, axis=0)
        self.stds = np.nanstd(X, axis=0)
        self.means = np.where(np.isnan(self.means), np.zeros(self.means.shape), self.means)
        self.stds = np.where(np.isnan(self.stds), np.ones(self.stds.shape), self.stds)
        self.stds = np.where(self.stds == 0, np.ones(self.stds.shape), self.stds)

        return self

    def transform(self, X: List[List[Optional[float]]]) -> np.ndarray:
        """
        Transforms the data by subtracting the means and dividing by the standard deviations.

        :param X: A list of lists of floats (or None).
        :return: The transformed data with NaNs replaced by :code:`self.replace_nan_token`.
        """
        X = np.array(X).astype(float)
        transformed_with_nan = (X - self.means) / self.stds
        transformed_with_none = np.where(np.isnan(transformed_with_nan), self.replace_nan_token, transformed_with_nan)

        return transformed_with_none

    def inverse_transform(self, X: List[List[Optional[float]]]) -> np.ndarray:
        """
        Performs the inverse transformation by multiplying by the standard deviations and adding the means.

        :param X: A list of lists of floats.
        :return: The inverse transformed data with NaNs replaced by :code:`self.replace_nan_token`.
        """
        X = np.array(X).astype(float)
        transformed_with_nan = X * self.stds + self.means
        transformed_with_none = np.where(np.isnan(transformed_with_nan), self.replace_nan_token, transformed_with_nan)

        return transformed_with_none.astype('float32')


def normalize_targets(y_data) -> StandardScaler:
    # For Future Use.
    """
    Normalizes the targets of the dataset using a :class:`~chemprop.data.StandardScaler`.

    The :class:`~chemprop.data.StandardScaler` subtracts the mean and divides by the standard deviation
    for each task independently.

    This should only be used for regression datasets.

    :return: A :class:`~chemprop.data.StandardScaler` fitted to the targets.
    """
    scaler = StandardScaler().fit(y_data)
    scaled_targets = scaler.transform(y_data).tolist()

    return scaled_targets, scaler

In [ ]:
seeds = [0,1,2,42,1234]
seed=seeds[1]
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)

In [ ]:
root_path = '/content/gdrive/MyDrive/function_predictor/GB1-Dataset-FewToMore'

In [ ]:
embedding_file = root_path+'/'+'emb_residue_level_embedding_GB1_embedding_ESM_2_650_embeddings_tensor.pt'
embeddings = torch.load(embedding_file)

In [ ]:
splits = ['low_vs_high', 'one_vs_rest', 'two_vs_rest', 'three_vs_rest']
split_name = splits[-1]
fasta_file = root_path+'/'+split_name+'.fasta'

In [ ]:
# [New] Parse FASTA file and create datasets
sequences = {}
with open(fasta_file, 'r') as file:
    for line in file:
        if line.startswith('>'):
            name, target, set_info, _ = line.strip().split(' ')
            name = name[1:]
            target = float(target.split('=')[1])
            set_type = set_info.split('=')[1]
            sequences[name] = {'target': target, 'set': set_type}

In [ ]:
train_data = []
test_data = []
for name, info in sequences.items():
    embedding = embeddings[f'{name}'].numpy()
    if info['set'] == 'train':
        train_data.append((embedding, info['target']))
    elif info['set'] == 'test':
        test_data.append((embedding, info['target']))

In [ ]:
len(train_data)

2990

In [ ]:
len(test_data)

5743

In [ ]:
from sklearn.preprocessing import StandardScaler as SklearnStandardScaler
class CNN_dataset(Dataset):
    def __init__(self, data, max_len, X_scaler=None):
        super().__init__()
        self.data = data
        self.max_len = max_len
        self.X_scaler = X_scaler

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        embedding, target = self.data[idx]

        # Reshape, scale and pad on-the-fly
        embedding = embedding.reshape(-1, 1280)
        if self.X_scaler is not None:
            embedding = self.X_scaler.transform(embedding)
        embedding = embedding.reshape(self.max_len, -1)
        padded_embedding = np.zeros((self.max_len, embedding.shape[1]))
        padded_embedding[:embedding.shape[0], :embedding.shape[1]] = embedding

        return {'seqs_embeddings': torch.from_numpy(padded_embedding), 'y_property': torch.tensor(target)}

def generate_CNN_loader(train_data, test_data, max_len, batch_size, scaler: Optional[StandardScaler] = None):
    # Fit the scaler if it's not already fitted
    X_tr = np.array([embedding.reshape(-1, 1280) for embedding, _ in train_data])
    X_tr = X_tr.reshape(-1, 1280)
    if scaler is None:
        scaler = SklearnStandardScaler()
    scaler.fit(X_tr)

    # Use the same dataset class for both train and test datasets
    train_dataset = CNN_dataset(train_data, max_len, scaler)
    test_dataset = CNN_dataset(test_data, max_len, scaler)

    # Create data loaders
    train_loader = DataLoader(train_dataset, batch_size, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size, shuffle=False)

    return train_loader, test_loader

In [ ]:
max_len = 265  # Set your sequence max length here
scaler = StandardScaler()  # Replace with any scaler you want
train_loader, test_loader = generate_CNN_loader(train_data, test_data, max_len, batch_size=256, scaler=scaler)

In [ ]:
class CNN(nn.Module):
    def __init__(self,
                 in_dim: int,
                 hid_dim: int,
                 kernal_1: int,
                 out_dim: int,
                 kernal_2: int,
                 max_len: int,
                 last_hid: int,
                 dropout: float = 0.
                 ):
        super().__init__()
        self.norm = nn.BatchNorm1d(in_dim)
        self.conv1 = nn.Conv1d(in_dim, hid_dim, kernal_1, padding=int((kernal_1-1)/2))
        self.dropout1 = nn.Dropout(dropout, inplace=True)
        #--------------------------------------------------#
        self.conv2_1 = nn.Conv1d(hid_dim, out_dim, kernal_2, padding=int((kernal_2-1)/2))
        self.dropout2_1 = nn.Dropout(dropout, inplace=True)
        #--------------------------------------------------#
        self.conv2_2 = nn.Conv1d(hid_dim, hid_dim, kernal_2, padding=int((kernal_2-1)/2))
        self.dropout2_2 = nn.Dropout(dropout, inplace=True)
        #--------------------------------------------------#
        self.conv3 = nn.Conv1d(hid_dim, out_dim, kernal_2, padding=int((kernal_2-1)/2))
        self.dropout3 = nn.Dropout(dropout, inplace=True)
        #self.pooling = nn.MaxPool1d(3, stride=3,padding=1)
        #--------------------------------------------------#
        self.fc_1 = nn.Linear(int(2*max_len*out_dim),last_hid)
        self.fc_2 = nn.Linear(last_hid,last_hid)
        self.fc_3 = nn.Linear(last_hid,1)
        self.cls = nn.Sigmoid()

    def forward(self, enc_inputs):
        #--------------------------------------------------#
        output = enc_inputs.transpose(1, 2)
        output = self.norm(output)
        output = nn.functional.relu(self.conv1(output))
        output = self.dropout1(output)
        #--------------------------------------------------#
        output_1 = nn.functional.relu(self.conv2_1(output))
        output_1 = self.dropout2_1(output_1)
        #--------------------------------------------------#
        output_2 = nn.functional.relu(self.conv2_2(output)) + output
        output_2 = self.dropout2_2(output_2)
        #--------------------------------------------------#
        output_2 = nn.functional.relu(self.conv3(output_2))
        output_2 = self.dropout3(output_2)
        #--------------------------------------------------#
        output = torch.cat((output_1,output_2),1)
        #print(output.size())
        #--------------------------------------------------#
        #output = self.pooling(output)
        #--------------------------------------------------#
        output = torch.flatten(output,1)
        #print(output.size())
        #--------------------------------------------------#
        output = self.fc_1(output)
        output = nn.functional.relu(output)
        output = self.fc_2(output)
        output = nn.functional.relu(output)
        output = self.fc_3(output)
        return output, output_1

In [ ]:
NN_input_dim = 1280 #embedding length
hid_dim    = 512    # 256
kernal_1   = 3      # 5
out_dim    = 1      # 2
kernal_2   = 3      # 3
last_hid   = 1024   # 1024
dropout    = 0.0
epoch_num      = 100
batch_size     = 256
learning_rate  =  [0.01        , # 0
                   0.005       , # 1
                   0.002       , # 2
                   0.001       , # 3
                   0.0005      , # 4
                   0.0002      , # 5
                   0.0001      , # 6
                   0.00005     , # 7
                   0.00002     , # 8
                   0.00001     , # 8
                   0.000005    , # 10
                   0.000002    , # 11
                   0.000001    , # 12
                   ][7]

In [ ]:
model = CNN(
            in_dim    =  NN_input_dim ,
            hid_dim   =  hid_dim      ,
            kernal_1  =  kernal_1     ,
            out_dim   =  out_dim      ,
            kernal_2  =  kernal_2     ,
            max_len   =  max_len ,
            last_hid  =  last_hid     ,
            dropout   =  dropout      ,
            )

model.float()
model.cuda()
#--------------------------------------------------#
print("#"*50)
print(model)
#model.float()
#print( summary( model,[(seqs_max_len, NN_input_dim),] )  )
#model.float()
print("#"*50)
#--------------------------------------------------#
optimizer = torch.optim.Adam(model.parameters(),lr=learning_rate)
#optimizer = torch.optim.SGD(model.parameters(),lr=learning_rate)
criterion = nn.MSELoss()

##################################################
CNN(
  (norm): BatchNorm1d(1280, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv1): Conv1d(1280, 512, kernel_size=(3,), stride=(1,), padding=(1,))
  (dropout1): Dropout(p=0.0, inplace=True)
  (conv2_1): Conv1d(512, 1, kernel_size=(3,), stride=(1,), padding=(1,))
  (dropout2_1): Dropout(p=0.0, inplace=True)
  (conv2_2): Conv1d(512, 512, kernel_size=(3,), stride=(1,), padding=(1,))
  (dropout2_2): Dropout(p=0.0, inplace=True)
  (conv3): Conv1d(512, 1, kernel_size=(3,), stride=(1,), padding=(1,))
  (dropout3): Dropout(p=0.0, inplace=True)
  (fc_1): Linear(in_features=530, out_features=1024, bias=True)
  (fc_2): Linear(in_features=1024, out_features=1024, bias=True)
  (fc_3): Linear(in_features=1024, out_features=1, bias=True)
  (cls): Sigmoid()
)
##################################################


In [ ]:
model.to(device)

CNN(
  (norm): BatchNorm1d(1280, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv1): Conv1d(1280, 512, kernel_size=(3,), stride=(1,), padding=(1,))
  (dropout1): Dropout(p=0.0, inplace=True)
  (conv2_1): Conv1d(512, 1, kernel_size=(3,), stride=(1,), padding=(1,))
  (dropout2_1): Dropout(p=0.0, inplace=True)
  (conv2_2): Conv1d(512, 512, kernel_size=(3,), stride=(1,), padding=(1,))
  (dropout2_2): Dropout(p=0.0, inplace=True)
  (conv3): Conv1d(512, 1, kernel_size=(3,), stride=(1,), padding=(1,))
  (dropout3): Dropout(p=0.0, inplace=True)
  (fc_1): Linear(in_features=530, out_features=1024, bias=True)
  (fc_2): Linear(in_features=1024, out_features=1024, bias=True)
  (fc_3): Linear(in_features=1024, out_features=1, bias=True)
  (cls): Sigmoid()
)

In [ ]:
def save_plot(y_pred, y_real, epoch, folder_path, file_name, fig_size=(12, 10), marker_size=10, fit_line_color="skyblue", distn_color_1="lightgreen", distn_color_2="salmon"):
    sns.set(style="whitegrid")

    # Create a jointplot
    g = sns.jointplot(
        x=y_real,
        y=y_pred,
        kind="reg",
        height=fig_size[1] - 1,
        color=fit_line_color,
        scatter_kws={"s": marker_size},
        marginal_kws={'color': distn_color_1}
    )

    # Set axis labels
    g.ax_joint.set_xlabel("Actual Values", fontsize=12)
    g.ax_joint.set_ylabel("Predictions", fontsize=12)

    # Set the title with a bit more space at the top
    g.fig.suptitle(f"Predictions vs. Actual Values\n R = {np.round(stats.pearsonr(y_pred, y_real)[0], 3)} | Epoch: {epoch}", fontsize=14, y=1.05)

    # Plot the histograms
    sns.histplot(y_real, color=distn_color_1, alpha=0.6, ax=g.ax_marg_x, fill=True, kde=True)
    sns.histplot(y=y_pred, color=distn_color_2, alpha=0.6, ax=g.ax_marg_y, fill=True, kde=True)

    # Adjust the plot margins and layout
    plt.subplots_adjust(left=0.15, right=0.85, top=0.85, bottom=0.15)
    g.fig.tight_layout()

    # Save the plot
    plt.savefig(Path(folder_path) / file_name, bbox_inches='tight')
    plt.close(g.fig)

In [21]:
# Training
model_name = 'cnn'
save_path = f'{root_path}/{model_name}_results/split_name_{split_name}/seed_{seed}'
test_interval = 5
best_r_score = -1
training_losses = []  # To store training losses for plotting

# Variables to keep track of the best file names
best_model_file = ""
best_plot_file = ""
best_metrics_file = ""
for epoch in range(epoch_num):
    begin_time = time.time()

    # Training Phase
    model.train()
    total_loss = 0
    for one_seq_ppt_group in train_loader:
        seq_rep, target = one_seq_ppt_group["seqs_embeddings"], one_seq_ppt_group["y_property"]
        seq_rep, target = seq_rep.float().to(device), target.float().to(device)


        optimizer.zero_grad()
        output, _ = model(seq_rep)
        loss = criterion(output, target.view(-1, 1))
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    avg_loss = total_loss / len(train_loader)
    training_losses.append(avg_loss)

    # Test Phase
    if epoch % test_interval == 0 or epoch == epoch_num - 1:
        model.eval()
        y_pred_test = []
        y_real_test = []
        for one_seq_ppt_group in test_loader:
            seq_rep, target = one_seq_ppt_group["seqs_embeddings"], one_seq_ppt_group["y_property"]
            seq_rep, target = seq_rep.float().to(device), target.float().to(device)

            output, _ = model(seq_rep)
            output = output.cpu().detach().numpy().reshape(-1)
            target = target.cpu().numpy()

            y_pred_test.extend(output)
            y_real_test.extend(target)

        _, _, r_value_test, _, _ = scipy.stats.linregress(y_pred_test, y_real_test)

        # Save best model
        if r_value_test >= best_r_score:
            best_r_score = r_value_test
            os.makedirs(save_path, exist_ok=True)
            if best_model_file:
                os.remove(best_model_file)
                os.remove(best_plot_file)
                os.remove(best_metrics_file)

            # Update file names
            best_model_file = f'{save_path}/best_model_epoch_{epoch}_seed_{seed}.pt'
            best_plot_file = f'{save_path}/test_plot_epoch_{epoch}.png'
            best_metrics_file = f'{save_path}/metrics_epoch_{epoch}.txt'
            torch.save(model.state_dict(), best_model_file)
            save_plot(y_pred_test, y_real_test, epoch, save_path, best_plot_file.split('/')[-1])
            with open(best_metrics_file, 'w') as f:
                f.write(f'Epoch: {epoch}\n')
                f.write(f'Test R-value: {r_value_test}\n')
                f.write(f'Test MAE: {mean_absolute_error(y_pred_test, y_real_test)}\n')
                f.write(f'Test MSE: {mean_squared_error(y_pred_test, y_real_test)}\n')
                f.write(f'Test RMSE: {np.sqrt(mean_squared_error(y_pred_test, y_real_test))}\n')
                f.write(f'Test R2: {r2_score(y_real_test, y_pred_test)}\n')
                f.write(f'Test Spearman rho: {scipy.stats.spearmanr(y_pred_test, y_real_test)[0]}\n')
                # Additional metrics can be added here
            print(f'Best R-value: {best_r_score:.4f}')

    print(f'Epoch: {epoch} | Training Loss: {avg_loss:.4f}')

Best R-value: 0.3359
Epoch: 0 | Training Loss: 1.6724
Epoch: 1 | Training Loss: 1.2096
Epoch: 2 | Training Loss: 1.1872
Epoch: 3 | Training Loss: 1.1767
Epoch: 4 | Training Loss: 1.1631
Best R-value: 0.5506
Epoch: 5 | Training Loss: 1.1552
Epoch: 6 | Training Loss: 1.1163
Epoch: 7 | Training Loss: 1.0695
Epoch: 8 | Training Loss: 1.0137
Epoch: 9 | Training Loss: 0.9204
Best R-value: 0.6069
Epoch: 10 | Training Loss: 0.8480
Epoch: 11 | Training Loss: 0.8126
Epoch: 12 | Training Loss: 0.7663
Epoch: 13 | Training Loss: 0.6922
Epoch: 14 | Training Loss: 0.6459
Best R-value: 0.6970
Epoch: 15 | Training Loss: 0.6054
Epoch: 16 | Training Loss: 0.5835
Epoch: 17 | Training Loss: 0.5620
Epoch: 18 | Training Loss: 0.5423
Epoch: 19 | Training Loss: 0.5178
Best R-value: 0.7406
Epoch: 20 | Training Loss: 0.5218
Epoch: 21 | Training Loss: 0.4864
Epoch: 22 | Training Loss: 0.4660
Epoch: 23 | Training Loss: 0.4584
Epoch: 24 | Training Loss: 0.4406
Best R-value: 0.7782
Epoch: 25 | Training Loss: 0.4525


In [22]:
# Save the training plot
plt.figure()
plt.plot(training_losses)
plt.xlabel('Epoch')
plt.ylabel('Training Loss')
plt.title('Training Loss Over Epochs')
plt.savefig(f'{save_path}/training_loss_plot_seed_{seed}.png')
plt.close()

print(f'Training Completed. Best Model Saved with R-score: {best_r_score}')

Training Completed. Best Model Saved with R-score: 0.8569787486198672
